In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests as rq
import time
import re

error = list()

mainUrl = 'https://www.myfooddiary.com'
directory = '/foods/brands'
url = f'{mainUrl}{directory}'

r = rq.get(url)
status = r.status_code # Estado de la conexion

if status != 200:
    print("Respuesta desconocida")
    quit()

soup = BeautifulSoup(r.content, 'html.parser')

# Crear la lista de comercios

brandsConteiner = soup.find("div", id="BrandListHldr") # Main container

brandMenuLinks = list() #Link list to all brands menus

for a in brandsConteiner.find_all("a"):
    link = f'{a["href"]}'
    brandMenuLinks.append({"brand":a.text , "href":link}) 
    #Por ahora lo voy a hacer asi pero mas delante con todos los datos tengo que hacer un diccionario por marca

# Recorrido de la lista de los menus de cada marca. El objetivo es capturar los items del menu

for i in brandMenuLinks[0:3]:
    # <------- Inicialmente trabajo con un slice de la lista para testear el loop ------->
    time.sleep(1)
    link = f'{mainUrl}{i["href"]}'
    r = rq.get(link)
    status = r.status_code
    soup = BeautifulSoup(r.content, 'html.parser')
    menuTable = soup.find("table", class_="FoodListTable")
    # Chequeo que la tabla como la busque exista
    if not menuTable == None:
        menuTable = menuTable.tbody
    else:
        error.append({"error":"No menuTable", "desc":"Error mientas buscaba la tabla con el menu"})
        continue
    menu = menuTable.find_all("tr")
    # Tomo todas las filas de la tabla, cada fila es un item en el menu
    if len(menu) > 0:
        menuItems = list()
        for p in menu:
            time.sleep(1)
            row = p.find("td", class_="FoodRowLt")
            if row == None:
                continue
            itemUrl = row.a["href"]
            itemName = row.a.span.next_sibling
            idP = row.a["id"]
            menuItems.append({"idp":idP, "itemName":itemName, "href":itemUrl})  
    i.update({"menu":menuItems}) # Actualizacion de los datos en el diccionario

# Proximo paso getNutritions facts


for i in brandMenuLinks[0:3]:
    # <------- Inicialmente trabajo con un slice de la lista para testear el loop ------->
    # por cada item del menu de cada restaurnte tengo un diccionario donde debo agregar la informacion nutricional
    # la informacion nutricional tambien va a ser una lista de diccionarios
    for item in i["menu"]:
        # -> CONEXION al item
        url = f'{mainUrl}{item["href"]}'
        r = rq.get(url)
        # Chequeo de status
        if r.status_code != 200:
            error.append({"error":f'Estado {r.status_code}', "desc":"Error mientas buscaba informacion nutricional"})
            continue
        # FIN CONEXION <-
        # -> INSTANCIA OBJETO SOUP
        soup = BeautifulSoup(r.content, "html.parser")
        # Aqui estan todos los datos
        foodLabel = soup.find("div", id="FoodLabelHldr")
# <---- Empiza la extraccion de los datos ----->
        netCarbs = foodLabel.find("div", id="nf_net_carbs_hldr").extract()
        calories = foodLabel.find("div",id="nf_cal_hldr").extract()
        if not netCarbs == None:
            try:
                netCarbs = [int(s) for s in netCarbs.text.split() if s.isdigit()][0]
            except :
                netCarbs = f'Fail {netCarbs}'
        else:
            netCarbs = "Not available"
        if not calories == None:
            try:
                calories = [int(s) for s in calories.text.split() if s.isdigit()][0]
            except :
                calories = f'Fail {calories}'
        # <- Salvo primeros datos -->
        #item.update({"nFacts":{"calories":calories, "netCarbs":netCarbs}})
        # <- end -->
        # <- Next -->
        # Los divs de interes tinenen un id con el siguiente formato
        patt = "nf_((?!amount)(?!dv_note)(?!daily))[a-zA-Z_]*"
        # Lista de todos los divs con los datos
        nfContainers = foodLabel.find_all("div", id=re.compile(patt))
        # Loop de elementos dentro del conetenedor de informacion nutricional
        nutrients = list()
        for nf in nfContainers:
            # print(f'**************')
            # Lista de tags en el contenedor
            tagsInNfC = nf.find_all(True) 
            if len(tagsInNfC):
            #    print(f'Contenedor Anidado ***********')
                nutrients.append(nFactData(nf))
            else:
                continue
        item.update({"nFacts":{"calories":calories, "netCarbs":netCarbs, "facts":nutrients}})

In [2]:
# Creacion de las tablas
data = list()
for i in brandMenuLinks[0:3]:
    marca = i["brand"]
    for it in i["menu"]:
        itemName = it["itemName"]
        itemId = it["idp"]
        itemUrl = it["href"]
        itemNetCarbs = it["nFacts"]["netCarbs"]
        itemCalories = it["nFacts"]["calories"]
        registro={
            "marca": marca, 
            "menuItem": itemName, 
            "itemId": itemId,  
            "itemUrl":itemUrl, 
            "netCarbs": itemNetCarbs, 
            "calories":itemCalories,
        }
        for nf in it["nFacts"]["facts"]:
            registro.update({f'{nf["nfName"]}':nf["amount"],f'{nf["nfName"]}-%':nf["dailyValue"]})
        data.append(registro)

In [3]:
pd.DataFrame(data)

,marca,menuItem,itemId,itemUrl,netCarbs,calories,total fat,total fat-%,saturated fat,saturated fat-%,...,dietary fiber,dietary fiber-%,sugars,sugars-%,protein,protein-%,total sugars,total sugars-%,added sugars,added sugars-%
0,18 Rabbits,"Date, Pecan & Coconut Granola Bar",Desc7190411,/foods/7190411/18-rabbits-date-pecan-coconut-g...,26,190,8g,12,3g,15,...,2g,7,11g,NaN,3g,None,NaN,NaN,NaN,NaN
1,18 Rabbits,"Cherry, Dark Chocolate & Almond Granola Bar",Desc7190410,/foods/7190410/18-rabbits-cherry-dark-chocolat...,26,200,9g,14,3.5g,18,...,2g,7,11g,NaN,3g,None,NaN,NaN,NaN,NaN
2,18 Rabbits,Chocolate & Cherry Jr. Granola Bar,Desc7190415,/foods/7190415/18-rabbits-chocolate-cherry-jr-...,18,130,4.5g,7,1g,5,...,2g,7,NaN,NaN,2g,None,7g,NaN,Includes 1g,2
3,18 Rabbits,"Almond, Pecan & Coconut Granola",Desc7517930,/foods/7517930/18-rabbits-almond-pecan-coconut...,13,150,9g,14,3.5g,18,...,2g,7,3g,NaN,3g,None,NaN,NaN,NaN,NaN
4,18 Rabbits,Chocolate & Banana Jr. Granola Bar,Desc7190417,/foods/7190417/18-rabbits-chocolate-banana-jr-...,18,130,5g,8,1g,5,...,2g,7,8g,NaN,2g,None,NaN,NaN,NaN,NaN
5,18 Rabbits,Mango Strawberry Jr. Granola Bar,Desc7190418,/foods/7190418/18-rabbits-mango-strawberry-jr-...,18,130,4g,6,0.5g,3,...,2g,7,NaN,NaN,2g,None,6g,NaN,Includes 1g,2
6,18 Rabbits,"Almond, Cashew & Coconut Grain Free Granola",Desc7517925,/foods/7517925/18-rabbits-almond-cashew-coconu...,6,188,16g,25,6g,30,...,3g,11,4g,NaN,5g,None,NaN,NaN,NaN,NaN
7,18 Rabbits,"Cashew, Amaranth & Cacao Nib Granola",Desc7517931,/foods/7517931/18-rabbits-cashew-amaranth-caca...,16,150,7g,11,2g,10,...,2g,7,4g,NaN,4g,None,NaN,NaN,NaN,NaN
8,18 Rabbits,"Cherry, Chia & Vanilla Granola",Desc7517932,/foods/7517932/18-rabbits-cherry-chia-vanilla-...,16,130,6g,9,0.5g,3,...,2g,7,4g,NaN,3g,None,NaN,NaN,NaN,NaN
9,180,Energy Drink – Orange Citrus Blast,Desc425273,/foods/425273/180-energy-drink-orange-citrus-b...,33,120,0g,0,0g,0,...,0g,0,30g,NaN,0g,None,NaN,NaN,NaN,NaN
